In [1]:
%%writefile decor_header.py
import random; from IPython.display import display,HTML
from IPython.core.magic import register_line_magic
@register_line_magic
def decor_header(params):
    params=params.split('|'); string=params[0]
    if len(params)==1: 
        font_size='22'; font_family='Wallpoet'; cmap='Sinebow'
    elif  len(params)==2: 
        font_size=params[1]
        font_family='Wallpoet'; cmap='Sinebow'
    elif  len(params)==3: 
        font_size=params[1]; font_family=params[2]
        cmap='Sinebow'
    else: 
        font_size=params[1]; font_family=params[2]; cmap=params[3]
    height=int(font_size)*3; randi=str(random.uniform(0,9999999))
    html_str="""
<script src='https://d3js.org/d3.v6.min.js'></script>
<style>
@import 'https://fonts.googleapis.com/css?family="""+font_family+"""';
#colorized001 {
font-family:"""+font_family+"""; font-size:"""+font_size+""";}
#canvas001,#canvas002 {width:10%; vertical-align:middle;}
</style>
<text id='colorized001'><canvas id='canvas001'></canvas>
"""+string+"""
<canvas id='canvas002'></canvas></text><br/>
<script>
var tc=setInterval(function() {
    var now=new Date().getTime();
    var iddoc=document.getElementById('colorized001');
    iddoc.style.color=d3.interpolate"""+cmap+"""(now/6000);
    var r=10,n=7;
    var c1=document.getElementById('canvas001'); 
    var context1=c1.getContext('2d');
    var c2=document.getElementById('canvas002'); 
    var context2=c2.getContext('2d');
    c1.style.background=d3.interpolate"""+cmap+"""(now/60000); 
    c2.style.background=d3.interpolate"""+cmap+"""(now/60000);
    context1.strokeStyle=d3.interpolate"""+cmap+"""(now/6000);    
    context2.strokeStyle=d3.interpolate"""+cmap+"""(now/6000);
    for (var i=1; i<n; i++) {
        context1.beginPath(); context2.beginPath();
        for (var j=0; j<6; j++) {
            context1.arc(60*j,r*(n+.5),i*r,0,2*Math.PI);
            context2.arc(60*j,r*(n+.5),i*r,0,2*Math.PI); };
        context1.stroke(); context2.stroke(); }; },1)
</script>"""
    file='d3header'+randi+'.html'
    with open(file,'w') as f:
         f.write(html_str); f.close()
    string="""<div id='html_string"""+randi+\
    """' style='width:100%;'><iframe src='"""+file+\
    """' height="""+str(height)+"""
    style='display:block; width:100%;'></iframe></div>"""
    display(HTML(string))

Writing decor_header.py


In [2]:
%run decor_header.py
%decor_header Modules, Helpful Functions, & Styling

In [3]:
!pip install --upgrade neural_structured_learning --user --quiet 

In [4]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Ewert|Roboto&effect=3d');
span {text-shadow:3px 3px 3px #aaa;}  
div.output_area pre{font-family:Roboto; font-size:110%; color:#ff36ff;}      
</style>

In [5]:
import warnings; warnings.filterwarnings('ignore')
import h5py,pylab as pl,pandas as pd
import numpy as np,tensorflow as tf
import neural_structured_learning as nsl
from sklearn.model_selection import train_test_split
fpath3='../input/classification-of-handwritten-letters/'
fpath4='../input/flower-color-images/'

In [6]:
def prepro(x_train,y_train,x_test,y_test,n_class):
    n=int(len(x_test)/2)    
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    cy_train=tf.keras.utils.to_categorical(y_train,n_class) 
    cy_valid=tf.keras.utils.to_categorical(y_valid,n_class)
    cy_test=tf.keras.utils.to_categorical(y_test,n_class)
    df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
                     [y_train.shape,y_valid.shape,y_test.shape],
                     [cy_train.shape,cy_valid.shape,cy_test.shape]],
                    columns=['train','valid','test'],
                    index=['images','labels','encoded labels'])
    display(df)
    return [[x_train,x_valid,x_test],
            [y_train,y_valid,y_test],
            [cy_train,cy_valid,cy_test]]

In [7]:
%decor_header Data Loading & Preprocessing

In [8]:
(x_train1,y_train1),(x_test1,y_test1)=\
tf.keras.datasets.mnist.load_data()
[[x_train1,x_valid1,x_test1],
 [y_train1,y_valid1,y_test1],
 [cy_train1,cy_valid1,cy_test1]]=\
prepro(x_train1/255,y_train1.reshape(-1,1),
       x_test1/255,y_test1.reshape(-1,1),10)

11493376/11490434 [==============================] - 0s 0us/step


,train,valid,test
images,"(60000, 28, 28)","(5000, 28, 28)","(5000, 28, 28)"
labels,"(60000, 1)","(5000, 1)","(5000, 1)"
encoded labels,"(60000, 10)","(5000, 10)","(5000, 10)"


In [9]:
(x_train2,y_train2),(x_test2,y_test2)=\
tf.keras.datasets.cifar10.load_data()
[[x_train2,x_valid2,x_test2],
 [y_train2,y_valid2,y_test2],
 [cy_train2,cy_valid2,cy_test2]]=\
prepro(x_train2/255,y_train2,x_test2/255,y_test2,10)

170500096/170498071 [==============================] - 3s 0us/step


,train,valid,test
images,"(50000, 32, 32, 3)","(5000, 32, 32, 3)","(5000, 32, 32, 3)"
labels,"(50000, 1)","(5000, 1)","(5000, 1)"
encoded labels,"(50000, 10)","(5000, 10)","(5000, 10)"


In [10]:
f=h5py.File(fpath3+'LetterColorImages_123.h5','r') 
keys=list(f.keys()); keys
images=np.array(f[keys[1]])/255
labels=np.array(f[keys[2]]).astype('int').reshape(-1,1)-1
x_train3,x_test3,y_train3,y_test3=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train3,x_valid3,x_test3],
 [y_train3,y_valid3,y_test3],
 [cy_train3,cy_valid3,cy_test3]]=\
prepro(x_train3,y_train3,x_test3,y_test3,33)

,train,valid,test
images,"(11352, 32, 32, 3)","(1419, 32, 32, 3)","(1419, 32, 32, 3)"
labels,"(11352, 1)","(1419, 1)","(1419, 1)"
encoded labels,"(11352, 33)","(1419, 33)","(1419, 33)"


In [11]:
f=h5py.File(fpath4+'FlowerColorImages.h5','r') 
keys=list(f.keys()); keys
images=np.array(f[keys[0]])/255
labels=np.array(f[keys[1]]).astype('int').reshape(-1,1)
x_train4,x_test4,y_train4,y_test4=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train4,x_valid4,x_test4],
 [y_train4,y_valid4,y_test4],
 [cy_train4,cy_valid4,cy_test4]]=\
prepro(x_train4,y_train4,x_test4,y_test4,10)

,train,valid,test
images,"(168, 128, 128, 3)","(21, 128, 128, 3)","(21, 128, 128, 3)"
labels,"(168, 1)","(21, 1)","(21, 1)"
encoded labels,"(168, 10)","(21, 10)","(21, 10)"


In [12]:
%decor_header Models with Adversarial Regularization
%decor_header MLP Base|18

In [13]:
batch_size=64; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size),name='input'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(n_class,activation=tf.nn.softmax)
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [14]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1,'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1,'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 938 steps, validate for 78 steps
Epoch 1/7
938/938 - 7s - loss: 0.2659 - sparse_categorical_crossentropy: 0.2140 - sparse_categorical_accuracy: 0.9345 - scaled_adversarial_loss: 0.0519 - val_loss: 0.2147 - val_sparse_categorical_crossentropy: 0.1727 - val_sparse_categorical_accuracy: 0.9433 - val_scaled_adversarial_loss: 0.0419
Epoch 2/7
938/938 - 5s - loss: 0.1145 - sparse_categorical_crossentropy: 0.0901 - sparse_categorical_accuracy: 0.9722 - scaled_adversarial_loss: 0.0244 - val_loss: 0.1983 - val_sparse_categorical_crossentropy: 0.1592 - val_sparse_categorical_accuracy: 0.9467 - val_scaled_adversarial_loss: 0.0391
Epoch 3/7
938/938 - 5s - loss: 0.0753 - sparse_categorical_crossentropy: 0.0579 - sparse_categorical_accuracy: 0.9824 - scaled_adversarial_loss: 0.0174 - val_loss: 0.2086 - val_sparse_categorical_crossentropy: 0.1672 - val_sparse_categorical_accuracy: 0.9525 - val_scaled_adversarial_loss: 0.0414
Epoch 4/7
938/938 - 5s - loss: 0.0521 - sparse_categorical_crossen

In [15]:
adv_model.evaluate({'input':x_test1,'label':y_test1})

5000/5000 [==============================] - 1s 205us/sample - loss: 0.0734 - sparse_categorical_crossentropy: 0.0579 - sparse_categorical_accuracy: 0.9852 - scaled_adversarial_loss: 0.0152


[0.07340541521113628, 0.05789365, 0.9852, 0.015161194]

In [16]:
batch_size=64; img_size=32; n_class=10; epochs=7
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [17]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 782 steps, validate for 78 steps
Epoch 1/7
782/782 - 9s - loss: 2.1252 - sparse_categorical_crossentropy: 1.7592 - sparse_categorical_accuracy: 0.3885 - scaled_adversarial_loss: 0.3660 - val_loss: 2.1954 - val_sparse_categorical_crossentropy: 1.8207 - val_sparse_categorical_accuracy: 0.3626 - val_scaled_adversarial_loss: 0.3747
Epoch 2/7
782/782 - 7s - loss: 1.7812 - sparse_categorical_crossentropy: 1.4745 - sparse_categorical_accuracy: 0.4748 - scaled_adversarial_loss: 0.3068 - val_loss: 2.0018 - val_sparse_categorical_crossentropy: 1.6580 - val_sparse_categorical_accuracy: 0.4295 - val_scaled_adversarial_loss: 0.3438
Epoch 3/7
782/782 - 7s - loss: 1.6452 - sparse_categorical_crossentropy: 1.3600 - sparse_categorical_accuracy: 0.5150 - scaled_adversarial_loss: 0.2852 - val_loss: 1.9839 - val_sparse_categorical_crossentropy: 1.6413 - val_sparse_categorical_accuracy: 0.4229 - val_scaled_adversarial_loss: 0.3426
Epoch 4/7
782/782 - 7s - loss: 1.5518 - sparse_categorical_crossen

In [18]:
adv_model.evaluate({'input':x_test2,'label':y_test2})

5000/5000 [==============================] - 1s 204us/sample - loss: 2.2364 - sparse_categorical_crossentropy: 1.8386 - sparse_categorical_accuracy: 0.4364 - scaled_adversarial_loss: 0.3959


[2.2364123416900634, 1.8385599, 0.4364, 0.39588258]

In [19]:
batch_size=128; img_size=32; n_class=33; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(33,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [20]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 89 steps, validate for 11 steps
Epoch 1/100
89/89 - 4s - loss: 3.3491 - sparse_categorical_crossentropy: 2.7251 - sparse_categorical_accuracy: 0.2359 - scaled_adversarial_loss: 0.6240 - val_loss: 5.6446 - val_sparse_categorical_crossentropy: 4.6884 - val_sparse_categorical_accuracy: 0.0646 - val_scaled_adversarial_loss: 0.9562
Epoch 2/100
89/89 - 1s - loss: 2.4188 - sparse_categorical_crossentropy: 1.9431 - sparse_categorical_accuracy: 0.4298 - scaled_adversarial_loss: 0.4757 - val_loss: 5.3691 - val_sparse_categorical_crossentropy: 4.4323 - val_sparse_categorical_accuracy: 0.1037 - val_scaled_adversarial_loss: 0.9369
Epoch 3/100
89/89 - 1s - loss: 1.9682 - sparse_categorical_crossentropy: 1.5671 - sparse_categorical_accuracy: 0.5297 - scaled_adversarial_loss: 0.4011 - val_loss: 4.7054 - val_sparse_categorical_crossentropy: 3.8586 - val_sparse_categorical_accuracy: 0.1875 - val_scaled_adversarial_loss: 0.8467
Epoch 4/100
89/89 - 1s - loss: 1.6481 - sparse_categorical_crossent

In [21]:
adv_model.evaluate({'input':x_test3,'label':y_test3})

1419/1419 [==============================] - 1s 417us/sample - loss: 2.2422 - sparse_categorical_crossentropy: 1.6914 - sparse_categorical_accuracy: 0.7357 - scaled_adversarial_loss: 0.5428


[2.2422102609934145, 1.6914164, 0.7357294, 0.5428185]

In [22]:
batch_size=16; img_size=128; n_class=10; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [23]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 11 steps, validate for 1 steps
Epoch 1/100
11/11 - 3s - loss: 3.8254 - sparse_categorical_crossentropy: 3.1716 - sparse_categorical_accuracy: 0.2321 - scaled_adversarial_loss: 0.6520 - val_loss: 8.3906 - val_sparse_categorical_crossentropy: 6.9898 - val_sparse_categorical_accuracy: 0.1875 - val_scaled_adversarial_loss: 1.4008
Epoch 2/100
11/11 - 0s - loss: 2.3404 - sparse_categorical_crossentropy: 1.9401 - sparse_categorical_accuracy: 0.5238 - scaled_adversarial_loss: 0.3927 - val_loss: 6.3208 - val_sparse_categorical_crossentropy: 5.2656 - val_sparse_categorical_accuracy: 0.2500 - val_scaled_adversarial_loss: 1.0552
Epoch 3/100
11/11 - 0s - loss: 0.6340 - sparse_categorical_crossentropy: 0.5249 - sparse_categorical_accuracy: 0.8036 - scaled_adversarial_loss: 0.1079 - val_loss: 4.5190 - val_sparse_categorical_crossentropy: 3.7639 - val_sparse_categorical_accuracy: 0.2500 - val_scaled_adversarial_loss: 0.7551
Epoch 4/100
11/11 - 0s - loss: 0.2039 - sparse_categorical_crossentr

In [24]:
adv_model.evaluate({'input':x_test4,'label':y_test4})

21/21 [==============================] - 0s 21ms/sample - loss: 2.2583 - sparse_categorical_crossentropy: 1.8708 - sparse_categorical_accuracy: 0.5238 - scaled_adversarial_loss: 0.3876


[2.258343458175659, 1.8707559, 0.52380955, 0.38758752]

In [25]:
%decor_header CNN Base|18

In [26]:
batch_size=128; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,1),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [27]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1.reshape(x_train1.shape[0],28,28,1),
     'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1.reshape(x_valid1.shape[0],28,28,1),
     'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 469 steps, validate for 39 steps
Epoch 1/7
469/469 - 10s - loss: 0.4617 - sparse_categorical_crossentropy: 0.3818 - sparse_categorical_accuracy: 0.8772 - scaled_adversarial_loss: 0.0799 - val_loss: 0.1597 - val_sparse_categorical_crossentropy: 0.1307 - val_sparse_categorical_accuracy: 0.9619 - val_scaled_adversarial_loss: 0.0290
Epoch 2/7
469/469 - 6s - loss: 0.1092 - sparse_categorical_crossentropy: 0.0895 - sparse_categorical_accuracy: 0.9735 - scaled_adversarial_loss: 0.0197 - val_loss: 0.0701 - val_sparse_categorical_crossentropy: 0.0569 - val_sparse_categorical_accuracy: 0.9832 - val_scaled_adversarial_loss: 0.0132
Epoch 3/7
469/469 - 6s - loss: 0.0811 - sparse_categorical_crossentropy: 0.0664 - sparse_categorical_accuracy: 0.9800 - scaled_adversarial_loss: 0.0146 - val_loss: 0.0578 - val_sparse_categorical_crossentropy: 0.0467 - val_sparse_categorical_accuracy: 0.9858 - val_scaled_adversarial_loss: 0.0111
Epoch 4/7
469/469 - 6s - loss: 0.0669 - sparse_categorical_crosse

In [28]:
adv_model.evaluate({'input':x_test1.reshape(x_test1.shape[0],28,28,1),
                    'label':y_test1})

5000/5000 [==============================] - 1s 258us/sample - loss: 0.0176 - sparse_categorical_crossentropy: 0.0140 - sparse_categorical_accuracy: 0.9946 - scaled_adversarial_loss: 0.0035


[0.01762931326192338, 0.014002147, 0.9946, 0.0035446528]

In [29]:
batch_size=64; img_size=32; n_class=10; epochs=20
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(96,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [30]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 782 steps, validate for 78 steps
Epoch 1/20
782/782 - 9s - loss: 1.9695 - sparse_categorical_crossentropy: 1.6360 - sparse_categorical_accuracy: 0.3975 - scaled_adversarial_loss: 0.3335 - val_loss: 1.6432 - val_sparse_categorical_crossentropy: 1.3591 - val_sparse_categorical_accuracy: 0.5162 - val_scaled_adversarial_loss: 0.2841
Epoch 2/20
782/782 - 8s - loss: 1.5665 - sparse_categorical_crossentropy: 1.2976 - sparse_categorical_accuracy: 0.5318 - scaled_adversarial_loss: 0.2689 - val_loss: 1.4569 - val_sparse_categorical_crossentropy: 1.2003 - val_sparse_categorical_accuracy: 0.5797 - val_scaled_adversarial_loss: 0.2566
Epoch 3/20
782/782 - 7s - loss: 1.4121 - sparse_categorical_crossentropy: 1.1681 - sparse_categorical_accuracy: 0.5830 - scaled_adversarial_loss: 0.2441 - val_loss: 1.3658 - val_sparse_categorical_crossentropy: 1.1219 - val_sparse_categorical_accuracy: 0.6052 - val_scaled_adversarial_loss: 0.2439
Epoch 4/20
782/782 - 8s - loss: 1.3166 - sparse_categorical_cro

In [31]:
adv_model.evaluate({'input':x_test2,'label':y_test2})

5000/5000 [==============================] - 1s 201us/sample - loss: 1.0260 - sparse_categorical_crossentropy: 0.8249 - sparse_categorical_accuracy: 0.7156 - scaled_adversarial_loss: 0.2015


[1.0260270330429078, 0.8248866, 0.7156, 0.20152642]

In [32]:
batch_size=128; img_size=32; n_class=33; epochs=150
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(33,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [33]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 89 steps, validate for 11 steps
Epoch 1/150
89/89 - 3s - loss: 4.1951 - sparse_categorical_crossentropy: 3.4959 - sparse_categorical_accuracy: 0.0307 - scaled_adversarial_loss: 0.6991 - val_loss: 4.1851 - val_sparse_categorical_crossentropy: 3.4874 - val_sparse_categorical_accuracy: 0.0320 - val_scaled_adversarial_loss: 0.6977
Epoch 2/150
89/89 - 1s - loss: 4.1458 - sparse_categorical_crossentropy: 3.4542 - sparse_categorical_accuracy: 0.0483 - scaled_adversarial_loss: 0.6916 - val_loss: 4.0808 - val_sparse_categorical_crossentropy: 3.3986 - val_sparse_categorical_accuracy: 0.0611 - val_scaled_adversarial_loss: 0.6822
Epoch 3/150
89/89 - 1s - loss: 4.0335 - sparse_categorical_crossentropy: 3.3583 - sparse_categorical_accuracy: 0.0674 - scaled_adversarial_loss: 0.6752 - val_loss: 4.0084 - val_sparse_categorical_crossentropy: 3.3365 - val_sparse_categorical_accuracy: 0.0788 - val_scaled_adversarial_loss: 0.6719
Epoch 4/150
89/89 - 1s - loss: 3.9653 - sparse_categorical_crossent

In [34]:
adv_model.evaluate({'input':x_test3,'label':y_test3})

1419/1419 [==============================] - 1s 476us/sample - loss: 0.4161 - sparse_categorical_crossentropy: 0.2999 - sparse_categorical_accuracy: 0.9161 - scaled_adversarial_loss: 0.1162


[0.41610196310504377, 0.29994494, 0.9161381, 0.116249464]

In [35]:
batch_size=16; img_size=32; n_class=10; epochs=100
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [36]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 11 steps, validate for 1 steps
Epoch 1/100
11/11 - 2s - loss: 2.7971 - sparse_categorical_crossentropy: 2.3295 - sparse_categorical_accuracy: 0.1131 - scaled_adversarial_loss: 0.4673 - val_loss: 2.7327 - val_sparse_categorical_crossentropy: 2.2767 - val_sparse_categorical_accuracy: 0.1250 - val_scaled_adversarial_loss: 0.4560
Epoch 2/100
11/11 - 0s - loss: 2.6584 - sparse_categorical_crossentropy: 2.2131 - sparse_categorical_accuracy: 0.2024 - scaled_adversarial_loss: 0.4452 - val_loss: 2.6086 - val_sparse_categorical_crossentropy: 2.1732 - val_sparse_categorical_accuracy: 0.1875 - val_scaled_adversarial_loss: 0.4354
Epoch 3/100
11/11 - 0s - loss: 2.3547 - sparse_categorical_crossentropy: 1.9604 - sparse_categorical_accuracy: 0.3869 - scaled_adversarial_loss: 0.3938 - val_loss: 2.4266 - val_sparse_categorical_crossentropy: 2.0211 - val_sparse_categorical_accuracy: 0.3125 - val_scaled_adversarial_loss: 0.4055
Epoch 4/100
11/11 - 0s - loss: 1.9645 - sparse_categorical_crossentr

In [37]:
adv_model.evaluate({'input':x_test4,'label':y_test4})

21/21 [==============================] - 0s 20ms/sample - loss: 0.8814 - sparse_categorical_crossentropy: 0.7228 - sparse_categorical_accuracy: 0.7619 - scaled_adversarial_loss: 0.1586


[0.8814017176628113, 0.7227542, 0.7619048, 0.15864752]